In [ ]:
#!/usr/bin/env python
#=========================================================================================
# 	This script converts .mat files to .nc
#	output files are written to odir
#	SciPy.io has a 'loadmat' function that is used for reading '.mat' files
#
#	The PyNIO (Nio) package is used for writing NetCDF files
#	and can be obtained from https://www.earthsystemgrid.org/home.htm
#
#    July, 2013	Walter Hannah 		University of Miami
#=========================================================================================
import os
import numpy as np
import scipy as sp
import scipy.io as spio
import Nio
#=========================================================================================
#=========================================================================================
debug    = True 	# writes to 'testfile' instead of to odir
verbose	 = False	# verbose output for debugging

testfile = 'test.nc'	# only for debug mode
idir = "/Users/.../matlabfiles/"		# input directory
odir = "/Users/.../ncfiles/"				# output directory

#define variables that will go into output NetCDF file
var_name = ["QGomega_PVA","QGomege_TA","QGomega_adia","QGomega_dia","QGomega_tot"]

yr = "2010"
m1 = 7
#=========================================================================================
#=========================================================================================

#---------------------------------------------------
# Create new NetCDF file for output
#---------------------------------------------------
ofile = odir+"QGomega."+yr+mn+dy+".nc"
if debug: ofile  = testfile
if os.path.isfile(ofile): os.remove(ofile)
outfile 		= Nio.open_file(ofile, "c")
outfile.history = "Ji Nie calculations, https://www.jiniepku.com/download.html"

hr = "{:0>2d}".format(h)

ifile = idir+"0000vars.mat"
matfile = spio.loadmat(ifile)

for key in matfile.keys():
    if type(matfile[key]).__name__ == 'ndarray':
        print "    "+key.ljust(15)+"  ",type(matfile[key]).__name__,"  ",matfile[key].shape
    else:
        print "    "+key.ljust(15)+"  ",type(matfile[key]).__name__

#---------------------------------------------------
# Create coordinate variables
#---------------------------------------------------
dims = matfile["QGomega"].shape

nlat = 
nlon = 
lat  = sp.linspace( matfile["minLat"][0,0], matfile["maxLat"][0,0], nlat )
lon  = sp.linspace( matfile["minLon"][0,0], matfile["maxLon"][0,0], nlon )

#---------------------------------------------------
# Write coordinate variables to output NetCDF file
#---------------------------------------------------

outfile.create_dimension('time', ntime)
time = sp.arange(ntime)
time = time.astype(np.int32, copy=False)		
ftime 		= outfile.create_variable('time', 'i', ('time',))
ftime[:]	= time
ftime.units = 'hours since '+yr+'-'+mn+'-'+dy+' 00:00:00'
del time


outfile.create_dimension('lon' , nlon)	
flon 		= outfile.create_variable('lon', 'd', ('lon',))
flon[:]	 	= lon
flon.units 	= 'Degrees East'


#---------------------------------------------------
# Transfer data from '.mat' files to output file
#---------------------------------------------------
tmp = matfile[var]
tmp = tmp.astype(np.float32, copy=False)

ftmp = outfile.create_variable(var, 'f', ('time','lat','lon'))
ftmp.units 		= 'mm'
ftmp.long_name	= "Total Precipitable Water"
ftmp[h,:,:]			= tmp
del tmp

#---------------------------------------------------
# Close the NetCDF file
#---------------------------------------------------
outfile.close()
#=========================================================================================
#=========================================================================================
